Installed biosimspace 2024.4.0 dev via
added gromacs and ambertools via conda-forge


In [1]:
import BioSimSpace as BSS

INFO:rdkit:Enabling RDKit 2024.03.5 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


In [2]:
protein_xtal = BSS.IO.readPDB("inputs/3STD_prep_fix.pdb", pdb4amber=True)[0]

Change view settings to highlight the xtal waters

In [3]:
view = BSS.Notebook.View(protein_xtal)

In [4]:
view.system()

ThemeManager()

NGLWidget(gui_style='ngl')

extract crystallographic waters

In [5]:
protein = protein_xtal.extract(
    [atom.index() for atom in protein_xtal.search("not resname HOH").atoms()]
)
water = protein_xtal.extract(
    [atom.index() for atom in protein_xtal.search("resname HOH").atoms()]
)

For the rest of the setup we are ignoring the xtal waters. Later we can check if GCMC found buried sites. The site of interest near the ligand is dry in the input

In [6]:
lig1 = BSS.IO.readMolecules("inputs/lig1.sdf")[0]

In [7]:
view = BSS.Notebook.View(lig1)

In [8]:
view.system()

NGLWidget(gui_style='ngl')

In [9]:
protein = BSS.Parameters.ff14SB(protein, work_dir="tmp").getMolecule()

In [10]:
lig1 = BSS.Parameters.openff_unconstrained_2_2_0(lig1).getMolecule()

In [11]:
complex = lig1 + protein

In [12]:
complex_sol = BSS.Solvent.tip3p(
    molecule=complex, box=3 * [60 * BSS.Units.Length.angstrom]
)

In [13]:
view = BSS.Notebook.View(complex_sol)

In [14]:
view.system()

NGLWidget(gui_style='ngl')

In [15]:
protocol = BSS.Protocol.Minimisation()
process = BSS.Process.OpenMM(complex_sol, protocol)
process.start()
process.wait()
minimised = process.getSystem()

In [16]:
protocol = BSS.Protocol.Equilibration(
    runtime=5 * BSS.Units.Time.picosecond,
    temperature_start=0 * BSS.Units.Temperature.kelvin,
    temperature_end=300 * BSS.Units.Temperature.kelvin,
    restraint="all",
)
process = BSS.Process.OpenMM(minimised, protocol)
process.start()
process.wait()
nvt = process.getSystem()

In [17]:
protocol = BSS.Protocol.Equilibration(
    runtime=20 * BSS.Units.Time.picosecond,
    pressure=1 * BSS.Units.Pressure.atm,
    temperature=300 * BSS.Units.Temperature.kelvin,
    restraint="heavy",
)
process = BSS.Process.OpenMM(nvt, protocol)
process.start()
process.wait()
npt = process.getSystem()

In [18]:
protocol = BSS.Protocol.Equilibration(
    runtime=75 * BSS.Units.Time.picosecond,
    pressure=1 * BSS.Units.Pressure.atm,
    temperature=300 * BSS.Units.Temperature.kelvin,
)
process = BSS.Process.OpenMM(npt, protocol)
process.start()
process.wait()
equilibrated = process.getSystem()

In [19]:
BSS.IO.saveMolecules("outputs/equilibrated", equilibrated, ["PDB", "RST7", "PRM7"])

['/home/director/GCMC/outputs/equilibrated.pdb',
 '/home/director/GCMC/outputs/equilibrated.rst7',
 '/home/director/GCMC/outputs/equilibrated.prm7']

In [20]:
BSS.Stream.save(equilibrated, "outputs/3STD-lig1")